In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 12 × Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, skylake)
  Threads: 1 on 12 virtual cores


## 6-2. イテレーションプロトコルの実装

### 6-2-1. サイズが決まっているパターン

#### 例1. 配列（`AbstractArray`）の派生型とする場合（推奨）

##### コード6-19. `GeometricArray`（多次元に対応した等比数列型）

```julia
struct GeometricArray{T<:Number, N} <: AbstractArray{T, N}
    a::T
    r::T
    ns::NTuple{N, Int}
end

function GeometricArray(a::T1, r::T2, ns::NTuple{N, Integer}) where {T1, T2, N}
    GeometricArray(promote(a, r)..., Int.(ns))
end

function Base.show(io::IO, box::GeometricArray)
    print(io, "GeometricArray(", box.a, ", ", box.r, ", ", box.ns, ")")
end
Base.show(io::IO, ::MIME"text/plain", box::GeometricArray) = show(io, box)

# Base.IteratorSize(GeometricArray{Int, 3}) == Base.HasShape{3}()

Base.size(box::GeometricArray) = box.ns
# `Base.length()` はデフォルト実装を利用

# Base.IteratorEltype(GeometricArray) == Base.HasEltype()
# eltype(GeometricArray{Int, 3}) == Int

Base.IndexStyle(::Type{<:GeometricArray}) = IndexLinear()
@inline function Base.getindex(box::GeometricArray, idx::Int) where {T, N}
    @boundscheck checkbounds(box, idx::Int)
    box.a * box.r ^ (idx - 1)
end
```

In [2]:
include("GeometricArray.jl")

##### コード6-20. `GeometricArray` の動作確認

In [3]:
garray3 = GeometricArray(10, 0.5, (5, 4, 3))

GeometricArray(10.0, 0.5, (5, 4, 3))

In [4]:
Base.IteratorSize(typeof(garray3))  # GeometricArray <: AbstractArray なので `HasShape{N}`

Base.HasShape{3}()

In [5]:
size(garray3)

(5, 4, 3)

In [6]:
length(garray3)

60

In [7]:
Base.IteratorEltype(typeof(garray3))  # GeometricArray <: AbstractArray なので `HasEleType`

Base.HasEltype()

In [8]:
Base.eltype(garray3)

Float64

In [9]:
for v in garray3
    println(v)
end

10.0
5.0
2.5
1.25
0.625
0.3125
0.15625
0.078125
0.0390625
0.01953125
0.009765625
0.0048828125
0.00244140625
0.001220703125
0.0006103515625
0.00030517578125
0.000152587890625
7.62939453125e-5
3.814697265625e-5
1.9073486328125e-5
9.5367431640625e-6
4.76837158203125e-6
2.384185791015625e-6
1.1920928955078125e-6
5.960464477539062e-7
2.980232238769531e-7
1.4901161193847656e-7
7.450580596923828e-8
3.725290298461914e-8
1.862645149230957e-8
9.313225746154785e-9
4.6566128730773926e-9
2.3283064365386963e-9
1.1641532182693481e-9
5.820766091346741e-10
2.9103830456733704e-10
1.4551915228366852e-10
7.275957614183426e-11
3.637978807091713e-11
1.8189894035458565e-11
9.094947017729282e-12
4.547473508864641e-12
2.2737367544323206e-12
1.1368683772161603e-12
5.684341886080801e-13
2.8421709430404007e-13
1.4210854715202004e-13
7.105427357601002e-14
3.552713678800501e-14
1.7763568394002505e-14
8.881784197001252e-15
4.440892098500626e-15
2.220446049250313e-15
1.1102230246251565e-15
5.551115123125783e-16
2.775

In [10]:
[v for v in garray3]

5×4×3 Array{Float64, 3}:
[:, :, 1] =
 10.0    0.3125     0.00976562   0.000305176
  5.0    0.15625    0.00488281   0.000152588
  2.5    0.078125   0.00244141   7.62939e-5
  1.25   0.0390625  0.0012207    3.8147e-5
  0.625  0.0195312  0.000610352  1.90735e-5

[:, :, 2] =
 9.53674e-6  2.98023e-7  9.31323e-9   2.91038e-10
 4.76837e-6  1.49012e-7  4.65661e-9   1.45519e-10
 2.38419e-6  7.45058e-8  2.32831e-9   7.27596e-11
 1.19209e-6  3.72529e-8  1.16415e-9   3.63798e-11
 5.96046e-7  1.86265e-8  5.82077e-10  1.81899e-11

[:, :, 3] =
 9.09495e-12  2.84217e-13  8.88178e-15  2.77556e-16
 4.54747e-12  1.42109e-13  4.44089e-15  1.38778e-16
 2.27374e-12  7.10543e-14  2.22045e-15  6.93889e-17
 1.13687e-12  3.55271e-14  1.11022e-15  3.46945e-17
 5.68434e-13  1.77636e-14  5.55112e-16  1.73472e-17

In [11]:
collect(garray3)

5×4×3 Array{Float64, 3}:
[:, :, 1] =
 10.0    0.3125     0.00976562   0.000305176
  5.0    0.15625    0.00488281   0.000152588
  2.5    0.078125   0.00244141   7.62939e-5
  1.25   0.0390625  0.0012207    3.8147e-5
  0.625  0.0195312  0.000610352  1.90735e-5

[:, :, 2] =
 9.53674e-6  2.98023e-7  9.31323e-9   2.91038e-10
 4.76837e-6  1.49012e-7  4.65661e-9   1.45519e-10
 2.38419e-6  7.45058e-8  2.32831e-9   7.27596e-11
 1.19209e-6  3.72529e-8  1.16415e-9   3.63798e-11
 5.96046e-7  1.86265e-8  5.82077e-10  1.81899e-11

[:, :, 3] =
 9.09495e-12  2.84217e-13  8.88178e-15  2.77556e-16
 4.54747e-12  1.42109e-13  4.44089e-15  1.38778e-16
 2.27374e-12  7.10543e-14  2.22045e-15  6.93889e-17
 1.13687e-12  3.55271e-14  1.11022e-15  3.46945e-17
 5.68434e-13  1.77636e-14  5.55112e-16  1.73472e-17

#### 例2. 配列（`AbstractArray`）の派生型としない場合

##### コード6-21. `GeometricBox`（多次元に対応した等比数列型（配列型の派生型としない実装））

```julia
struct GeometricBox{T<:Number, N}
    a::T
    r::T
    ns::NTuple{N, Int}
end

function GeometricBox(a::T1, r::T2, ns::NTuple{N, Integer}) where {T1, T2, N}
    GeometricBox(promote(a, r)..., Int.(ns))
end

#### `IteratorSize()`
Base.IteratorSize(::Type{GeometricBox{T, N}}) where {T, N} = Base.HasShape{N}()
Base.size(box::GeometricBox) = box.ns
Base.length(box::GeometricBox) = prod(box.ns)

#### `IteratorEltype()`
# Base.IteratorEltype(::Type{<:GeometricBox}) = Base.HasEltype()
Base.eltype(::Type{<:GeometricBox{T}}) where {T} = T

#### `iterate()`
Base.iterate(box::GeometricBox) = (box.a, (box.a, length(box) - 1))
function Base.iterate(box::GeometricBox{T}, (prev, rest)::Tuple{T, Int}) where {T}
    rest == 0 && return nothing
    next = prev * box.r
    (next, (next, rest - 1))
end
```

In [12]:
include("GeometricBox.jl")

##### コード6-22. `GeometricBox` の動作確認

In [13]:
gbox3 = GeometricBox(10, 0.5, (5, 4, 3))

GeometricBox{Float64, 3}(10.0, 0.5, (5, 4, 3))

In [14]:
Base.IteratorSize(typeof(gbox3))

Base.HasShape{3}()

In [15]:
size(gbox3)

(5, 4, 3)

In [16]:
length(gbox3)

60

In [17]:
Base.IteratorEltype(typeof(gbox3))

Base.HasEltype()

In [18]:
Base.eltype(gbox3)

Float64

In [19]:
for v in gbox3
    println(v)
end

10.0
5.0
2.5
1.25
0.625
0.3125
0.15625
0.078125
0.0390625
0.01953125
0.009765625
0.0048828125
0.00244140625
0.001220703125
0.0006103515625
0.00030517578125
0.000152587890625
7.62939453125e-5
3.814697265625e-5
1.9073486328125e-5
9.5367431640625e-6
4.76837158203125e-6
2.384185791015625e-6
1.1920928955078125e-6
5.960464477539062e-7
2.980232238769531e-7
1.4901161193847656e-7
7.450580596923828e-8
3.725290298461914e-8
1.862645149230957e-8
9.313225746154785e-9
4.6566128730773926e-9
2.3283064365386963e-9
1.1641532182693481e-9
5.820766091346741e-10
2.9103830456733704e-10
1.4551915228366852e-10
7.275957614183426e-11
3.637978807091713e-11
1.8189894035458565e-11
9.094947017729282e-12
4.547473508864641e-12
2.2737367544323206e-12
1.1368683772161603e-12
5.684341886080801e-13
2.8421709430404007e-13
1.4210854715202004e-13
7.105427357601002e-14
3.552713678800501e-14
1.7763568394002505e-14
8.881784197001252e-15
4.440892098500626e-15
2.220446049250313e-15
1.1102230246251565e-15
5.551115123125783e-16
2.775

In [20]:
[v for v in gbox3]

5×4×3 Array{Float64, 3}:
[:, :, 1] =
 10.0    0.3125     0.00976562   0.000305176
  5.0    0.15625    0.00488281   0.000152588
  2.5    0.078125   0.00244141   7.62939e-5
  1.25   0.0390625  0.0012207    3.8147e-5
  0.625  0.0195312  0.000610352  1.90735e-5

[:, :, 2] =
 9.53674e-6  2.98023e-7  9.31323e-9   2.91038e-10
 4.76837e-6  1.49012e-7  4.65661e-9   1.45519e-10
 2.38419e-6  7.45058e-8  2.32831e-9   7.27596e-11
 1.19209e-6  3.72529e-8  1.16415e-9   3.63798e-11
 5.96046e-7  1.86265e-8  5.82077e-10  1.81899e-11

[:, :, 3] =
 9.09495e-12  2.84217e-13  8.88178e-15  2.77556e-16
 4.54747e-12  1.42109e-13  4.44089e-15  1.38778e-16
 2.27374e-12  7.10543e-14  2.22045e-15  6.93889e-17
 1.13687e-12  3.55271e-14  1.11022e-15  3.46945e-17
 5.68434e-13  1.77636e-14  5.55112e-16  1.73472e-17

In [21]:
collect(gbox3)

5×4×3 Array{Float64, 3}:
[:, :, 1] =
 10.0    0.3125     0.00976562   0.000305176
  5.0    0.15625    0.00488281   0.000152588
  2.5    0.078125   0.00244141   7.62939e-5
  1.25   0.0390625  0.0012207    3.8147e-5
  0.625  0.0195312  0.000610352  1.90735e-5

[:, :, 2] =
 9.53674e-6  2.98023e-7  9.31323e-9   2.91038e-10
 4.76837e-6  1.49012e-7  4.65661e-9   1.45519e-10
 2.38419e-6  7.45058e-8  2.32831e-9   7.27596e-11
 1.19209e-6  3.72529e-8  1.16415e-9   3.63798e-11
 5.96046e-7  1.86265e-8  5.82077e-10  1.81899e-11

[:, :, 3] =
 9.09495e-12  2.84217e-13  8.88178e-15  2.77556e-16
 4.54747e-12  1.42109e-13  4.44089e-15  1.38778e-16
 2.27374e-12  7.10543e-14  2.22045e-15  6.93889e-17
 1.13687e-12  3.55271e-14  1.11022e-15  3.46945e-17
 5.68434e-13  1.77636e-14  5.55112e-16  1.73472e-17

### 6-2-2. 長さが不定のパターン

#### 例. コラッツの予想

##### コード6-23. Collatz（コラッツの予想をイテレーションで実現する例）

```julia
struct Collatz{I <: Integer}
    start::I
end

Base.IteratorSize(::Type{<:Collatz}) = Base.SizeUnknown()

Base.IteratorEltype(::Type{<:Collatz}) = Base.HasEltype()
Base.eltype(::Type{Collatz{I}}) where I = I

Base.iterate(c::Collatz) = (c.start, c.start)
function Base.iterate(::Collatz, prev)
    prev == 1 && return nothing
    next = if iseven(prev)
        prev ÷ 2
    else
        3prev + 1
    end
    (next, next)
end
```

In [22]:
include("Collatz.jl");

##### コード6-24. コラッツの予想（長さが有限のものの確認）

In [23]:
for n in Collatz(3)
    println(n)
end

3
10
5
16
8
4
2
1


In [24]:
collect(Collatz(27))

112-element Vector{Int64}:
  27
  82
  41
 124
  62
  31
  94
  47
 142
  71
 214
 107
 322
   ⋮
  53
 160
  80
  40
  20
  10
   5
  16
   8
   4
   2
   1

In [25]:
maximum(Collatz(27))

9232

##### コード6-25. コラッツの予想（初期値が負数のもの）

Do not execute below:

```julia
julia> for n in Collatz(-3)
           println(n)
       end
-3
-8
-4
-2
-1
-2
-1
-2
-1
-2
-1
-2
-1
 : 《以下延々と繰り返し…》
```

In [26]:
# collect(Collatz(-3))  # ←実行してはいけません！
collect(Iterators.take(Collatz(-3), 20))

20-element Vector{Int64}:
 -3
 -8
 -4
 -2
 -1
 -2
 -1
 -2
 -1
 -2
 -1
 -2
 -1
 -2
 -1
 -2
 -1
 -2
 -1
 -2

### 6-2-3. 無限列挙

#### 例. フィボナッチ数列

##### コード6-26. フィボナッチ数列

```julia
struct FibSequence end
const Fib = FibSequence()

Base.IteratorSize(::Type{FibSequence}) = Base.IsInfinite()

# Base.IteratorEltype(::Type{FibSequence}) = Base.HasEltype()
Base.eltype(::Type{FibSequence}) = BigInt

Base.iterate(::FibSequence) = iterate(Fib, (big"1", big"0"))

function Base.iterate(::FibSequence, (Fₙ₋₂, Fₙ₋₁)::NTuple{2, BigInt})
    Fₙ = Fₙ₋₂ + Fₙ₋₁
    (Fₙ, (Fₙ₋₁, Fₙ))
end
```

In [27]:
include("FibSequence.jl");

##### コード6-27. フィボナッチ数列の動作確認

In [28]:
for n in Iterators.take(Fib, 10)
    println(n)
end

1
1
2
3
5
8
13
21
34
55


In [29]:
collect(Iterators.take(Fib, 20))

20-element Vector{BigInt}:
    1
    1
    2
    3
    5
    8
   13
   21
   34
   55
   89
  144
  233
  377
  610
  987
 1597
 2584
 4181
 6765

In [30]:
first(Iterators.drop(Fib, 99))  # the 100th number

354224848179261915075

In [31]:
collect(Fib)  # `Base.IsInfinite()` ならばエラーとなる

LoadError: MethodError: no method matching _collect(::UnitRange{Int64}, ::FibSequence, ::Base.HasEltype, ::Base.IsInfinite)
[0mClosest candidates are:
[0m  _collect(::Any, ::Any, ::Base.HasEltype, [91m::Union{Base.HasLength, Base.HasShape}[39m) at array.jl:718
[0m  _collect(::Any, ::Any, ::Base.HasEltype, [91m::Base.SizeUnknown[39m) at array.jl:721
[0m  _collect(::Any, ::Any, [91m::Base.EltypeUnknown[39m, [91m::Union{Base.HasLength, Base.HasShape}[39m) at array.jl:804
[0m  ...